In [ ]:
%load_ext line_profiler

from pathlib import Path
import sys

import numpy as np

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [ ]:
import pandas as pd
import src.pipeline.rag as rag
import glob 
import os
import logging
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Loading saved comparator sets

In [ ]:
raw_data = pd.read_parquet('output/comparator-sets/academies.parquet')
raw_comparators = pd.read_parquet('output/comparator-sets/academy_comparators.parquet')

In [ ]:
raw_data.loc[147756]

In [ ]:
comps = raw_comparators.loc[135665]["Pupil"]
series = raw_data[raw_data.index.isin(comps)]

(df, categories) = rag.get_category_cols_predicates("Teaching and Teaching support staff", series)
categories

Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [ ]:
rs = rag.compute_rag(raw_data, raw_comparators)
pd.DataFrame(rs)

In [ ]:
%lprun -f rag.find_percentile list(rag.compute_rag(raw_data.head(10), raw_comparators))